# LSTM을 이용한 텍스트 생성
- 데이터 다운로드: 캐글 https://www.kaggle.com/aashita/nyt-comments
- ArticlesApril2018.csv

In [1]:
import numpy as np
import tensorflow as tf
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 파일 업로드

In [3]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving ArticlesApril2018.csv to ArticlesApril2018 (1).csv


### 데이터 전처리

In [4]:
import pandas as pd
df = pd.read_csv(filename, encoding='latin1')
df.head(2)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...


In [5]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [6]:
# headline 열의 데이터만 사용, Null 검사
df['headline'].isnull().values.any()

False

In [7]:
# headline 열의 데이터로 리스트 생성
headline = [title for title in df.headline.values]
headline[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [8]:
len(headline)

1324

In [9]:
# 노이즈 데이터('Unknown') 제거
headline = [title for title in headline if title != 'Unknown']
len(headline)

1214

In [10]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
# 구두점 제거와 소문자화를 위한 함수
def preprocessing(s):
    s=s.encode("utf8").decode("ascii", 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [12]:
headlines = [preprocessing(x) for x in headline]
headlines[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [13]:
# 단어 집합(vocabulary)을 만들고 크기를 확인
t = Tokenizer()
t.fit_on_texts(headlines)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [14]:
type(t.word_index)

dict

In [15]:
sequences = []
for line in headlines: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [16]:
index_to_word = {}
for key, value in t.word_index.items():     # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 1번 단어:', index_to_word[1])
print('빈도수 상위 582번 단어:', index_to_word[582])

빈도수 상위 1번 단어: the
빈도수 상위 582번 단어: offer


In [17]:
max_len = max(len(s) for s in sequences)
print(f'샘플의 최대 길이: {max_len}')

샘플의 최대 길이: 24


In [18]:
# 전체 샘플의 길이를 24(가장 긴 샘플의 길이)로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [19]:
# 각 샘플의 마지막 단어를 레이블로 분리
X = sequences[:,:-1]
y = sequences[:,-1]
# 리스트의 마지막 값을 제외하고 저장한 것은 X
# 리스트의 마지막 값만 저장한 것은 y. 이는 레이블에 해당됨.

In [20]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
Y = to_categorical(y, num_classes=vocab_size)

In [21]:
X.shape, Y.shape

((7803, 23), (7803, 3494))

### 모델 정의/설정/학습

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GRU

In [23]:
# 임베딩 벡터는 10차원, LSTM 노드 크기는 128
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1, 
                    name="Embedding_Layer"))
model.add(GRU(128, name="GRU_Layer"))
model.add(Dense(vocab_size, activation='softmax', name="Output_Layer"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 23, 10)            34940     
_________________________________________________________________
GRU_Layer (GRU)              (None, 128)               53760     
_________________________________________________________________
Output_Layer (Dense)         (None, 3494)              450726    
Total params: 539,426
Trainable params: 539,426
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [25]:
history = model.fit(X, Y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 3s - loss: 7.7057 - accuracy: 0.0291
Epoch 2/200
244/244 - 1s - loss: 7.1229 - accuracy: 0.0315
Epoch 3/200
244/244 - 1s - loss: 6.9182 - accuracy: 0.0432
Epoch 4/200
244/244 - 1s - loss: 6.7109 - accuracy: 0.0495
Epoch 5/200
244/244 - 1s - loss: 6.4869 - accuracy: 0.0534
Epoch 6/200
244/244 - 1s - loss: 6.2371 - accuracy: 0.0556
Epoch 7/200
244/244 - 1s - loss: 5.9657 - accuracy: 0.0624
Epoch 8/200
244/244 - 1s - loss: 5.6905 - accuracy: 0.0679
Epoch 9/200
244/244 - 1s - loss: 5.4227 - accuracy: 0.0759
Epoch 10/200
244/244 - 1s - loss: 5.1652 - accuracy: 0.0834
Epoch 11/200
244/244 - 1s - loss: 4.9260 - accuracy: 0.0951
Epoch 12/200
244/244 - 1s - loss: 4.6989 - accuracy: 0.1153
Epoch 13/200
244/244 - 1s - loss: 4.4823 - accuracy: 0.1402
Epoch 14/200
244/244 - 1s - loss: 4.2784 - accuracy: 0.1656
Epoch 15/200
244/244 - 1s - loss: 4.0816 - accuracy: 0.2011
Epoch 16/200
244/244 - 1s - loss: 3.8975 - accuracy: 0.2302
Epoch 17/200
244/244 - 1s - loss: 3.7178 - accura

### 모델 검증

In [26]:
def sentence_generation(model, t, current_word, n):     # 모델, 토크나이저, 현재 단어, 반복 횟수
    init_word = current_word
    sentence = ''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=23, padding='pre')
        #result = model.predict_classes(encoded, verbose=0)
        result = np.argmax(model.predict(encoded), axis=-1)
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
    
    sentence = init_word + sentence
    return sentence

In [27]:
print(sentence_generation(model, t, 'i', 10))
print(sentence_generation(model, t, 'how', 10))
print(sentence_generation(model, t, 'former', 10))

i want to be rich and im not sorry to help
how do you feel about being told to smile up and
former nfl cheerleaders settlement offer 1 and a meeting with goodell


### SimpleRNN을 사용한 경우

In [28]:
from tensorflow.keras.layers import SimpleRNN
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1, 
                    name="Embedding_Layer"))
model.add(SimpleRNN(128, name="SimpleRNN_Layer"))
model.add(Dense(vocab_size, activation='softmax', name="Output_Layer"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 23, 10)            34940     
_________________________________________________________________
SimpleRNN_Layer (SimpleRNN)  (None, 128)               17792     
_________________________________________________________________
Output_Layer (Dense)         (None, 3494)              450726    
Total params: 503,458
Trainable params: 503,458
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

In [30]:
history = model.fit(X, Y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 4s - loss: 7.6650 - accuracy: 0.0296
Epoch 2/200
244/244 - 3s - loss: 7.1293 - accuracy: 0.0319
Epoch 3/200
244/244 - 4s - loss: 6.9232 - accuracy: 0.0423
Epoch 4/200
244/244 - 3s - loss: 6.6457 - accuracy: 0.0527
Epoch 5/200
244/244 - 3s - loss: 6.3117 - accuracy: 0.0574
Epoch 6/200
244/244 - 3s - loss: 5.9754 - accuracy: 0.0647
Epoch 7/200
244/244 - 3s - loss: 5.6466 - accuracy: 0.0741
Epoch 8/200
244/244 - 3s - loss: 5.3306 - accuracy: 0.0847
Epoch 9/200
244/244 - 3s - loss: 5.0323 - accuracy: 0.1092
Epoch 10/200
244/244 - 3s - loss: 4.7332 - accuracy: 0.1402
Epoch 11/200
244/244 - 3s - loss: 4.4521 - accuracy: 0.1810
Epoch 12/200
244/244 - 3s - loss: 4.1840 - accuracy: 0.2302
Epoch 13/200
244/244 - 3s - loss: 3.9291 - accuracy: 0.2693
Epoch 14/200
244/244 - 3s - loss: 3.6971 - accuracy: 0.3099
Epoch 15/200
244/244 - 3s - loss: 3.4753 - accuracy: 0.3526
Epoch 16/200
244/244 - 3s - loss: 3.2726 - accuracy: 0.3861
Epoch 17/200
244/244 - 3s - loss: 3.0834 - accura

In [31]:
print(sentence_generation(model, t, 'i', 10))
print(sentence_generation(model, t, 'how', 10))
print(sentence_generation(model, t, 'former', 10))

i want to be rich and im not sorry spy at
how trump misunderstands trade case on choose a end to weapons
former judge to review files seized in searches ok in corruption


### LSTM
- i want to be rich and im not sorry border wild
- how to make a crossword puzzle wages with the people from
- former nfl cheerleaders settlement offer 1 and a meeting with goodell

### GRU
- i want to be rich and im not sorry to help
- how do you feel about being told to smile up and
- former nfl cheerleaders settlement offer 1 and a meeting with goodell

### SimpleRNN
- i want to be rich and im not sorry spy at
- how trump misunderstands trade case on choose a end to weapons
- former judge to review files seized in searches ok in corruption